# Lab 7: Self-Attention


This lab covers the following topics:

- Gain insight into the self-attention operation using the sequential MNIST example from before.
- Gain insight into positional encodings

## 0 Initialization

Run the code cell below to download the MNIST digits dataset:

In [78]:
!wget -O MNIST.tar.gz https://activeeon-public.s3.eu-west-2.amazonaws.com/datasets/MNIST.new.tar.gz
!tar -zxvf MNIST.tar.gz

import torchvision
import torch
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as F

from torch.utils.data import Subset


dataset = torchvision.datasets.MNIST('./', download=False, transform=transforms.Compose([transforms.ToTensor()]), train=True)
train_indices = torch.arange(0, 10000)
train_dataset = Subset(dataset, train_indices)

dataset=torchvision.datasets.MNIST('./', download=False, transform=transforms.Compose([transforms.ToTensor()]), train=False)
test_indices = torch.arange(0, 10000)
test_dataset = Subset(dataset, test_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
                                          shuffle=True, num_workers=0)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16,
                                          shuffle=False, num_workers=0)

--2023-03-09 18:46:16--  https://activeeon-public.s3.eu-west-2.amazonaws.com/datasets/MNIST.new.tar.gz
Resolving activeeon-public.s3.eu-west-2.amazonaws.com (activeeon-public.s3.eu-west-2.amazonaws.com)... 52.95.143.90
Connecting to activeeon-public.s3.eu-west-2.amazonaws.com (activeeon-public.s3.eu-west-2.amazonaws.com)|52.95.143.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34812527 (33M) [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz        100%[===================>]  33.20M   759KB/s    in 46s     

2023-03-09 18:47:03 (738 KB/s) - ‘MNIST.tar.gz’ saved [34812527/34812527]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/train-images-idx3-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/processed/
MNIST/processed/test.pt
MNIST/processed/training.pt


## Exercise 1: Self-Attention without Positional Encoding

In this section, will implement a very simple model based on self-attention without positional encoding. The model you will implement will consider the input image as a sequence of 28 rows. You may use PyTorch's [`nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) for this part. Implement a model with the following architecture:

* **Input**: Input image of shape `(batch_size, sequence_length, input_size)`, where $\text{sequence_length} = \text{image_height}$ and $\text{input_size} = \text{image_width}$.

* **Linear 1**: Linear layer which converts input of shape `(sequence_length*batch_size, input_size)` to input of shape `(sequence_length*batch_size, embed_dim)`, where `embed_dim` is the embedding dimension.

* **Attention 1**: `nn.MultiheadAttention` layer with 8 heads which takes an input of shape `(sequence_length, batch_size, embed_dim)` and outputs a tensor of shape `(sequence_length, batch_size, embed_dim)`. 

* **ReLU**: ReLU activation layer.

* **Linear 2**: Linear layer which converts input of shape `(sequence_length*batch_size, embed_dim)` to input of shape `(sequence_length*batch_size, embed_dim)`.

* **ReLU**: ReLU activation layer.

* **Attention 2**: `nn.MultiheadAttention` layer with 8 heads which takes an input of shape `(sequence_length, batch_size, embed_dim)` and outputs a tensor of shape `(sequence_length, batch_size, embed_dim)`.

* **ReLU**: ReLU activation layer.

* **AvgPool**: Average along the sequence dimension from `(batch_size, sequence_length, embed_dim)` to `(batch_size, embed_dim)`

* **Linear 3**: Linear layer which takes an input of shape `(batch_size, embed_dim)` and outputs the class logits of shape `(batch_size, 10)`.


**NOTE**: Be cautious of correctly permuting and reshaping the input between layers. E.g. if `x` is of shape `(batch_size, sequence_length, input_size)`, note that `x.reshape(batch_size*sequence_length, -1) != x.permute(1,0,2).reshape(batch_size*sequence_length, -1)`. In this example, `x.reshape(batch_size*sequence_length, -1)` has `[batch0_seq0, batch0_seq1, ..., batch1_seq0, batch1_seq1, ...]` format, while `x.permute(1,0,2).reshape(batch_size*sequence_length, -1)` has `[batch0_seq0, batch1_seq0, ..., batch0_seq1, batch1_seq1, ...]` format.

In [79]:
# Self-attention without positional encoding
torch.manual_seed(691)

# Define your model here
class myModel(nn.Module):
    def __init__(self, input_size, embed_dim, seq_length,
                 num_classes=10, num_heads=8):
        super(myModel, self).__init__()
        # TODO: Initialize myModel
        self.linear1 = nn.Linear(input_size, embed_dim)
        self.attention1 = nn.MultiheadAttention(embed_dim, num_heads)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(embed_dim, embed_dim)
        self.attention2 = nn.MultiheadAttention(embed_dim, num_heads)
        self.avgpool = nn.AvgPool1d(kernel_size=seq_length)
        self.linear3 = nn.Linear(embed_dim, num_classes)
        

    def forward(self,x):
        # TODO: Implement myModel forward pass
        batch_size, sequence_length, input_size = x.shape
        x = self.linear1(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embed_dim)
        x, _ = self.attention1(x, x, x)
        x = x.permute(1, 0, 2)  # (batch_size, sequence_length, embed_dim)
        x = self.relu(x)
        
        x = self.linear2(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embed_dim)
        x, _ = self.attention2(x, x, x)
        x = x.permute(1, 0, 2)  # (batch_size, sequence_length, embed_dim)
        x = self.relu(x)
        
        x = x.permute(0, 2, 1)  # (batch_size, sequence_length, embed_dim)
        x = self.avgpool(x) 
        x = x.squeeze()
        x = self.linear3(x)
        
        return x


Train and evaluate your model by running the cell below. Expect to see  `60-80%` test accuracy.

In [81]:
# Same training code 

import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 64
num_layers = 2
num_classes = 10
num_epochs = 8
learning_rate = 0.005

# Initialize model
model = myModel(input_size=input_size, embed_dim=hidden_size, seq_length=sequence_length)
model = model.to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Epoch [1/8], Step [10/157], Loss: 2.2624
Epoch [1/8], Step [20/157], Loss: 2.1277
Epoch [1/8], Step [30/157], Loss: 2.1047
Epoch [1/8], Step [40/157], Loss: 1.9855
Epoch [1/8], Step [50/157], Loss: 1.9740
Epoch [1/8], Step [60/157], Loss: 1.9792
Epoch [1/8], Step [70/157], Loss: 1.8781
Epoch [1/8], Step [80/157], Loss: 1.7266
Epoch [1/8], Step [90/157], Loss: 1.8464
Epoch [1/8], Step [100/157], Loss: 2.0648
Epoch [1/8], Step [110/157], Loss: 1.7438
Epoch [1/8], Step [120/157], Loss: 1.7033
Epoch [1/8], Step [130/157], Loss: 1.5998
Epoch [1/8], Step [140/157], Loss: 1.6166
Epoch [1/8], Step [150/157], Loss: 1.4920
Epoch [2/8], Step [10/157], Loss: 1.3090
Epoch [2/8], Step [20/157], Loss: 1.2383
Epoch [2/8], Step [30/157], Loss: 1.1932
Epoch [2/8], Step [40/157], Loss: 1.2777
Epoch [2/8], Step [50/157], Loss: 1.1584
Epoch [2/8], Step [60/157], Loss: 1.4048
Epoch [2/8], Step [70/157], Loss: 1.1303
Epoch [2/8], Step [80/157], Loss: 1.1631
Epoch [2/8], Step [90/157], Loss: 1.1413
Epoch [2/8

## Exercise 2: Self-Attention with Positional Encoding

Implement a similar model to exercise 1, except this time your embedded input should be added with the positional encoding. For the purpose of this lab, we will use a learned positional encoding, which will be a trainable embedding. Your positional encodings will be added to the initial transformation of the input.

* **Input**: Input image of shape `(batch_size, sequence_length, input_size)`, where $\text{sequence_length} = \text{image_height}$ and $\text{input_size} = \text{image_width}$.

* **Linear 1**: Linear layer which converts input of shape `(batch_size*sequence_length, input_size)` to input of shape `(batch_size*sequence_length, embed_dim)`, where `embed_dim` is the embedding dimension.

* **Add Positional Encoding**: Add a learnable positional encoding of shape `(sequence_length, batch_size, embed_dim)` to input of shape `(sequence_length, batch_size, embed_dim)`, where `pos_embed` is the positional embedding size. The output will be of shape `(sequence_length, batch_size, embed_dim)`.

* **Attention 1**: `nn.MultiheadAttention` layer with 8 heads which takes an input of shape `(sequence_length, batch_size, embed_dim)` and outputs a tensor of shape `(sequence_length, batch_size, embed_dim)`.

* **ReLU**: ReLU activation layer.

* **Linear 2**: Linear layer which converts input of shape `(sequence_length*batch_size, features_dim)` to input of shape `(sequence_length*batch_size, features_dim)`.

* **ReLU**: ReLU activation layer.

* **Attention 2**: `nn.MultiheadAttention` layer with 8 heads which takes an input of shape `(sequence_length, batch_size, features_dim)` and outputs a tensor of shape `(sequence_length, batch_size, features_dim)`.

* **ReLU**: ReLU activation layer.

* **AvgPool**: Average along the sequence dimension from `(batch_size, sequence_length, features_dim)` to `(batch_size, features_dim)`

* **Linear 3**: Linear layer which takes an input of shape `(batch_size, sequence_length*features_dim)` and outputs the class logits of shape `(batch_size, 10)`.


In [86]:
# Self-attention with positional encoding
torch.manual_seed(691)

# Define your model here
class myModel(nn.Module):
    def __init__(self, input_size, embed_dim, seq_length,
                 num_classes=10, num_heads=8):
        super(myModel, self).__init__()
        # TODO: Initialize myModel
        self.positional_encoding = nn.Parameter(torch.rand(seq_length, embed_dim))
        self.linear1 = nn.Linear(input_size, embed_dim)
        self.attention1 = nn.MultiheadAttention(embed_dim, num_heads)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(embed_dim, embed_dim)
        self.attention2 = nn.MultiheadAttention(embed_dim, num_heads)
        self.avgpool = nn.AvgPool1d(kernel_size=seq_length)
        self.linear3 = nn.Linear(embed_dim, num_classes)
        

    def forward(self,x):
        # TODO: Implement myModel forward pass
        batch_size, sequence_length, input_size = x.shape
        x = self.linear1(x)
        x = x + self.positional_encoding
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embed_dim)
        x, _ = self.attention1(x, x, x)
        x = x.permute(1, 0, 2)  # (batch_size, sequence_length, embed_dim)
        x = self.relu(x)
        
        x = self.linear2(x)
        x = x.permute(1, 0, 2)  # (sequence_length, batch_size, embed_dim)
        x, _ = self.attention2(x, x, x)
        x = x.permute(1, 0, 2)  # (batch_size, sequence_length, embed_dim)
        x = self.relu(x)
        
        x = x.permute(0, 2, 1)  # (batch_size, sequence_length, embed_dim)
        x = self.avgpool(x) 
        x = x.squeeze()
        x = self.linear3(x)
        
        return x

Use the same training code as the one from part 1 to train your model. You may copy the training loop here. Expect to see close to `~90+%` test accuracy.

In [87]:
# Same training code 

import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 64
num_layers = 2
num_classes = 10
num_epochs = 8
learning_rate = 0.005

# Initialize model
model = myModel(input_size=input_size, embed_dim=hidden_size, seq_length=sequence_length)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Epoch [1/8], Step [10/157], Loss: 2.3532
Epoch [1/8], Step [20/157], Loss: 2.2670
Epoch [1/8], Step [30/157], Loss: 2.0717
Epoch [1/8], Step [40/157], Loss: 1.9044
Epoch [1/8], Step [50/157], Loss: 1.3539
Epoch [1/8], Step [60/157], Loss: 1.5157
Epoch [1/8], Step [70/157], Loss: 0.9950
Epoch [1/8], Step [80/157], Loss: 0.6726
Epoch [1/8], Step [90/157], Loss: 0.9412
Epoch [1/8], Step [100/157], Loss: 0.8562
Epoch [1/8], Step [110/157], Loss: 0.7320
Epoch [1/8], Step [120/157], Loss: 0.8684
Epoch [1/8], Step [130/157], Loss: 1.1102
Epoch [1/8], Step [140/157], Loss: 0.7787
Epoch [1/8], Step [150/157], Loss: 0.4449
Epoch [2/8], Step [10/157], Loss: 0.5150
Epoch [2/8], Step [20/157], Loss: 0.3155
Epoch [2/8], Step [30/157], Loss: 0.4993
Epoch [2/8], Step [40/157], Loss: 0.5215
Epoch [2/8], Step [50/157], Loss: 0.4435
Epoch [2/8], Step [60/157], Loss: 0.6631
Epoch [2/8], Step [70/157], Loss: 0.3215
Epoch [2/8], Step [80/157], Loss: 0.3125
Epoch [2/8], Step [90/157], Loss: 0.2238
Epoch [2/8